In [91]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
import math
from copy import deepcopy

In [103]:
def normalize(x, mmin, mmax):
    return (x - mmin) / (mmax - mmin)


def scale_array(x, mmin=0, mmax=1, new_min=-1, new_max=1):
    x = normalize(x, mmin, mmax)
    return x * (new_max - new_min) + new_min


folder = 'train_data'
training_files = glob.glob(folder + '/*.csv')

first = True
X_full = y_full = None

for f in training_files:
    # read dataset
    train_ds = pd.read_csv(f, header=None)

    X = train_ds.iloc[:, :-4].values
    y = train_ds.iloc[:, -4:].values

    X_full = X if first else np.concatenate((X_full, X))
    y_full = y if first else np.concatenate((y_full, y))

    first = False

# Fill missing values with mean
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
imputer = imputer.fit(X_full)
X_full = imputer.transform(X_full)

In [104]:
for i in np.arange(X_full.shape[1]):
    print('{} - min: {}; max: {}'.format(i+1, X_full[:, i].min(), X_full[:, i].max()))

1 - min: -66.9462; max: 277.22700000000003
2 - min: -105.065; max: 143.624
3 - min: -3.14154; max: 3.1415
4 - min: -1.0; max: 6.0
5 - min: 0.0; max: 10472.0
6 - min: -46.0786; max: 233.389
7 - min: -57.6023; max: 232.908
8 - min: -57.5049; max: 278.655
9 - min: -56.3168; max: 278.39
10 - min: -1.0; max: 200.0
11 - min: -1.0; max: 200.0
12 - min: -1.0; max: 200.0
13 - min: -1.0; max: 200.0
14 - min: -1.0; max: 200.0
15 - min: -1.0; max: 200.0
16 - min: -1.0; max: 200.0
17 - min: -1.0; max: 200.0
18 - min: -1.0; max: 200.0
19 - min: -1.0; max: 200.0
20 - min: -1.0; max: 200.0
21 - min: -1.0; max: 200.0
22 - min: -1.0; max: 200.0
23 - min: -1.0; max: 200.0
24 - min: -1.0; max: 200.0
25 - min: -1.0; max: 200.0
26 - min: -1.0; max: 200.0
27 - min: -1.0; max: 200.0
28 - min: -1.0; max: 200.0


In [108]:
# Min and max values for specific parameters
maxSpeedX = X_full[:, 0].max()
minSpeedX = X_full[:, 0].min()
maxSpeedY = X_full[:, 1].max()
minSpeedY = X_full[:, 1].min()
maxRPM = X_full[:, 4].max()
maxWheelSpin = [X_full[:, i].max() for i in np.arange(5, 9)]
minWheelSpin = [X_full[:, i].min() for i in np.arange(5, 9)]
minDistFromEdge = X_full[:, 9:].min()

param_dict = {
    'maxSpeedX': maxSpeedX,
    'minSpeedX': minSpeedX,
    'maxSpeedY': maxSpeedY,
    'minSpeedY': minSpeedY,
    'maxRPM': maxRPM,
    'maxWheelSpin': maxWheelSpin,
    'minWheelSpin': minWheelSpin,
    'minDistFromEdge': minDistFromEdge
}

X_train = np.zeros(X_full.shape)

X_train[:, 0] = scale_array(X_full[:, 0], minSpeedX, maxSpeedX)
X_train[:, 1] = scale_array(deepcopy(X_full[:, 1]), minSpeedY, maxSpeedY)
X_train[:, 2] = scale_array(deepcopy(np.clip(X_full[:, 2], -math.pi, math.pi)), -math.pi, math.pi)

gears = deepcopy(X_full[:, 3])
gears[gears > 6] = 6
gears[gears < -1] = -1
X_train[:, 3] = scale_array(gears, -1, 6)

X_train[:, 4] = scale_array(deepcopy(np.clip(X_full[:, 4], 0, maxRPM)), 0, maxRPM)

for i in np.arange(5, 9):
    X_train[:, i] = scale_array(deepcopy(X_full[:, i]), minWheelSpin[i-5], maxWheelSpin[i-5])
for i in np.arange(9, 28):
    X_train[:, i] = scale_array(deepcopy(X_full[:, i]), minDistFromEdge, 200)

# y_train = np.zeros(y_full.shape)

# # gear = range(-1, 6)
# gears = y_full[:, 0]
# gears[gears == 7] = 6
# y_train[:, 0] = normalize_to_int(gears, -1, 6)
# # steering = range(-1, 1)
# y_train[:, 1] = np.clip(y_full[:, 1], -1, 1)
# # accelerate-brake = range(-1, 1)
# accel_brake = y_full[:, 2] - y_full[:, 3]
# y_train[:, 2] = normalize_to_int(accel_brake, -1, 1)

# y_train = np.delete(y_train, 3, axis=1)

In [109]:
for i in np.arange(X_train.shape[1]):
    print('{} - min: {}; max: {}'.format(i+1, X_train[:, i].min(), X_train[:, i].max()))

1 - min: -1.0; max: 1.0
2 - min: -1.0; max: 1.0
3 - min: -0.9999832398418258; max: 0.9999705074463787
4 - min: -1.0; max: 1.0
5 - min: -1.0; max: 1.0
6 - min: -1.0; max: 1.0
7 - min: -1.0; max: 1.0
8 - min: -1.0; max: 1.0
9 - min: -1.0; max: 1.0
10 - min: -1.0; max: 1.0
11 - min: -1.0; max: 1.0
12 - min: -1.0; max: 1.0
13 - min: -1.0; max: 1.0
14 - min: -1.0; max: 1.0
15 - min: -1.0; max: 1.0
16 - min: -1.0; max: 1.0
17 - min: -1.0; max: 1.0
18 - min: -1.0; max: 1.0
19 - min: -1.0; max: 1.0
20 - min: -1.0; max: 1.0
21 - min: -1.0; max: 1.0
22 - min: -1.0; max: 1.0
23 - min: -1.0; max: 1.0
24 - min: -1.0; max: 1.0
25 - min: -1.0; max: 1.0
26 - min: -1.0; max: 1.0
27 - min: -1.0; max: 1.0
28 - min: -1.0; max: 1.0
